# GAT with CORA Dataset

###installing spektral

In [3]:
!pip install --quiet spektral

##importing dependencies

In [4]:
"""
This example implements the experiments on citation networks from the paper:
Graph Attention Networks (https://arxiv.org/abs/1710.10903)
Petar Veličković, Guillem Cucurull, Arantxa Casanova, Adriana Romero, Pietro Liò, Yoshua Bengio
"""
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.random import set_seed

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GATConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess

set_seed(0)

## Load Data

In [5]:
# Load data
dataset = Citation(
    "cora", normalize_x=True, transforms=[LayerPreprocess(GATConv), AdjToSpTensor()]
)


Pre-processing node features


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


### Defining masks

In [6]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

### Defining Parameters

In [7]:
# Parameters
channels = 8  # Number of channels in each head of the first GAT layer
n_attn_heads = 8  # Number of attention heads in first GAT layer
dropout = 0.6  # Dropout rate for the features and adjacency matrix
l2_reg = 2.5e-4  # L2 regularization rate
learning_rate = 5e-3  # Learning rate
epochs = 20000  # Number of training epochs
patience = 100  # Patience for early stopping

N = dataset.n_nodes  # Number of nodes in the graph
F = dataset.n_node_features  # Original size of node features
n_out = dataset.n_labels  # Number of classes

## Defining the Model

In [8]:
# Model definition
x_in = Input(shape=(F,))
a_in = Input((N,), sparse=True)

do_1 = Dropout(dropout)(x_in)
gc_1 = GATConv(
    channels,
    attn_heads=n_attn_heads,
    concat_heads=True,
    dropout_rate=dropout,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_1, a_in])
do_2 = Dropout(dropout)(gc_1)
gc_2 = GATConv(
    n_out,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=dropout,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([do_2, a_in])

### Compiling and Building the Model

In [9]:
# Build model
model = Model(inputs=[x_in, a_in], outputs=gc_2)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer,
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
gat_conv (GATConv)              (None, 64)           91904       dropout[0][0]                    
                                                                 input_2[0][0]                

## Training

In [10]:
# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/20000
1/1 [==============================] - 3s 3s/step - loss: 1.9498 - acc: 0.1929 - val_loss: 1.9509 - val_acc: 0.1820
Epoch 2/20000
1/1 [==============================] - 0s 171ms/step - loss: 1.9472 - acc: 0.2143 - val_loss: 1.9505 - val_acc: 0.1260
Epoch 3/20000
1/1 [==============================] - 0s 179ms/step - loss: 1.9450 - acc: 0.2929 - val_loss: 1.9486 - val_acc: 0.2380
Epoch 4/20000
1/1 [==============================] - 0s 181ms/step - loss: 1.9447 - acc: 0.3000 - val_loss: 1.9461 - val_acc: 0.1860
Epoch 5/20000
1/1 [==============================] - 0s 168ms/step - loss: 1.9373 - acc: 0.4071 - val_loss: 1.9436 - val_acc: 0.2040
Epoch 6/20000
1/1 [==============================] - 0s 175ms/step - loss: 1.9363 - acc: 0.3714 - val_loss: 1.9413 - val_acc: 0.2100
Epoch 7/20000
1/1 [==============================] - 0s 181ms/step - loss: 1.9369 - acc: 0.3357 - val_loss: 1.9390 - val_acc: 0.3000
Epoch 8/20000
1/1 [==============================] - 0s 174ms/step - los

## Evaluation

In [11]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 54ms/step - loss: 0.9697 - acc: 0.8330
Done.
Test loss: 0.9697448015213013
Test accuracy: 0.833000123500824
